In [1]:
from perses.app.relative_setup import RelativeFEPSetup
import simtk.unit as unit
from perses.annihilation.relative import HybridTopologyFactory
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
from openmmtools import mcmc, utils
import openmmtools.integrators as integrators
import numpy as np

DEBUG:numba.interpreter:label 0:
    a = arg(0, name=a)                       ['a']
    b = arg(1, name=b)                       ['b']
    $0.1 = global(np: <module 'numpy' from '/home/dominic/.local/lib/python3.6/site-packages/numpy-1.17.2-py3.6-linux-x86_64.egg/numpy/__init__.py'>) ['$0.1']
    $0.2 = getattr(value=$0.1, attr=zeros)   ['$0.1', '$0.2']
    $const0.3 = const(int, 3)                ['$const0.3']
    $0.4 = call $0.2($const0.3, func=$0.2, args=[Var($const0.3, coordinate_numba.py:6)], kws=(), vararg=None) ['$0.2', '$0.4', '$const0.3']
    c = $0.4                                 ['$0.4', 'c']
    $const0.6 = const(int, 1)                ['$const0.6']
    $0.7 = getitem(value=a, index=$const0.6) ['$0.7', '$const0.6', 'a']
    $const0.9 = const(int, 2)                ['$const0.9']
    $0.10 = getitem(value=b, index=$const0.9) ['$0.10', '$const0.9', 'b']
    $0.11 = $0.7 * $0.10                     ['$0.10', '$0.11', '$0.7']
    $const0.13 = const(int, 2)               ['$co

In [2]:
from openmmtools.utils import get_available_platforms
plats = get_available_platforms()
for plat in plats:
    print(plat.getName())
    
import openmmtools.cache as cache
from perses.dispersed.utils import configure_platform
print(cache.global_context_cache.platform.getName())

Reference
CPU
conducting subsequent work with the following platform: CPU
CPU


In [3]:
molecule_setup = RelativeFEPSetup(ligand_input = 'test.smi', 
                                  old_ligand_index = 3, 
                                  new_ligand_index = 4, 
                                  forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'], 
                                  phases = ['solvent'],
                                  protein_pdb_filename=None,
                                  receptor_mol2_filename=None, 
                                  pressure=1.0 * unit.atmosphere,
                                  temperature=300.0 * unit.kelvin, 
                                  solvent_padding=9.0 * unit.angstroms, 
                                  atom_map=None,
                                  hmass=4*unit.amus, 
                                  neglect_angles=False, 
                                  map_strength='default', 
                                  anneal_14s = False)

INFO:relative_setup:			_hmass: 4 Da.

INFO:relative_setup:Handling files for ligands and indices...
INFO:relative_setup:Detected .smi format.  Proceeding...
INFO:relative_setup:	old smiles: CCCO
INFO:relative_setup:	new smiles: CCCS
INFO:utils.openeye:molecule  does not have unique atom names. Generating now...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:utils.openeye:molecule  does not have unique atom names. Generating now...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:relative_setup:	successfully created old and new systems from smiles
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:propan-1-ol
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule

added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.09903398679072904), ('CustomTorsionForce', 0.002959054139482202), ('CustomBondForce', 1.9604030488167976)]


INFO:geometry:	final no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	there are 3 bond forces in the no-nonbonded final system
INFO:geometry:	there are 19 angle forces in the no-nonbonded final system
INFO:geometry:	there are 22 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 1.7890162426022216
INFO:geometry:total reduced energy added from growth system: -1.7818771939630165
INFO:geometry:final reduced energy 0.00713904863920553
INFO:geometry:sum of energies: 0.0071390486392051145
INFO:geometry:magnitude of difference in the energies: 4.440892098500626e-16
INFO:geometry:Final logp_proposal: 3.1913713094213128


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.0023465627603357813), ('CustomTorsionForce', 6.390202536223275e-05), ('CustomBondForce', -1.7842876587487142)]


In [4]:
htf = HybridTopologyFactory(molecule_setup._solvent_topology_proposal,
                            molecule_setup._ligand_positions_old_solvated,
                            molecule_setup._ligand_positions_new_solvated)

In [5]:
from perses.tests.utils import validate_endstate_energies
ENERGY_THRESHOLD = 1e-6

from openmmtools.constants import kB
temperature = 300 * unit.kelvin

In [6]:
zero_state_error, one_state_error = validate_endstate_energies(htf._topology_proposal, htf, molecule_setup._solvated_added_valence_energy, molecule_setup._solvated_subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)

			HarmonicBondForce: 0.3302988765866683
			HarmonicAngleForce: 0.17577870393365402
			PeriodicTorsionForce: 1.2853491268675972
			NonbondedForce: -245.0384418652845
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-243.24701515789658
rp: -242.97027844182267
			CustomBondForce: 0.3302988765866683
			HarmonicBondForce: 0.0
			CustomAngleForce: 0.17343214117331823
			HarmonicAngleForce: 0.10138054955106482
			CustomTorsionForce: 1.285285224842235
			PeriodicTorsionForce: 0.003022956164844435
			NonbondedForce: -235.69070772433176
			CustomNonbondedForce: -7.387331092134983
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-241.1846190681486
rp: -240.90788235207472
			CustomBondForce: 64.3572998501014
			HarmonicBondForce: 0.0
			CustomAngleForce: 0.7988136664411729
			HarmonicAngleForce: 0.10138054955106482
			CustomTorsionForce: 0.010659379893790519
			PeriodicTorsionForce: 0.003022956164844435
			NonbondedForce: -218.3449242488

so it looks like we can properly maintain an endstate validation with small molecules, but what about protein mutations; is this the reason we are having issues with protein mutations?

anyway, at this point, let's just run this and see if it goes fast...

In [7]:
def create_splitting_integrator(htf, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """
    fast_lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
    fast_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))
    
    fast_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature = temperature),composable_states = [fast_lambda_alchemical_state])
    
    fast_sampler_state = SamplerState(positions = htf._hybrid_positions, box_vectors = htf.hybrid_system.getDefaultPeriodicBoxVectors())
    
#     integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
#                                                      timestep = 0.5* unit.femtoseconds,
#                                                      splitting = 'V R O R V',
#                                                      measure_shadow_work = False,
#                                                      measure_heat = False,
#                                                      constraint_tolerance = constraint_tol,
#                                                      collision_rate = 5.0 / unit.picoseconds)
    mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep = 4.0 * unit.femtoseconds,
                                                             collision_rate=5.0 / unit.picosecond,
                                                             n_steps=1,
                                                             reassign_velocities=False,
                                                             n_restart_attempts=20,
                                                             splitting="V R O R V",
                                                             constraint_tolerance=constraint_tol)
    
    
    #print(integrator_1.getConstraintTolerance())
    
#     fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
    
    
#     fast_sampler_state.apply_to_context(fast_context)
    
    return mcmc_moves, fast_thermodynamic_state, fast_sampler_state


    
def create_langevin_integrator(htf, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """
    fast_lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
    fast_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))
    
    fast_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature = temperature),composable_states = [fast_lambda_alchemical_state])
    
    fast_sampler_state = SamplerState(positions = htf._hybrid_positions, box_vectors = htf.hybrid_system.getDefaultPeriodicBoxVectors())
    
    integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
                                                     timestep = 4.0* unit.femtoseconds,
                                                     splitting = 'V R O R V',
                                                     measure_shadow_work = False,
                                                     measure_heat = False,
                                                     constraint_tolerance = constraint_tol,
                                                     collision_rate = 5.0 / unit.picoseconds)
#     mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep = 4.0 * unit.femtoseconds,
#                                                              collision_rate=5.0 / unit.picosecond,
#                                                              n_steps=1,
#                                                              reassign_velocities=False,
#                                                              n_restart_attempts=20,
#                                                              splitting="V R R R O R R R V",
#                                                              constraint_tolerance=constraint_tol)
    
    
    #print(integrator_1.getConstraintTolerance())
    
    fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
    
    
    fast_sampler_state.apply_to_context(fast_context)
    
    return fast_context, fast_thermodynamic_state, fast_sampler_state, fast_integrator    
    
    

In [8]:
def splitting_integrator_timer(move, thermostate, sstate, num_steps = 10):
    import time
    _time = []
    move.apply(thermostate, sstate)
    for i in range(num_steps):
        start = time.time()
        #integrator.step(1)
        move.apply(thermostate, sstate)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

In [9]:
def langevin_integrator_timer(integrator, num_steps = 10):
    import time
    _time = []
    integrator.step(1)
    #move.apply(thermostate, sstate)
    for i in range(num_steps):
        start = time.time()
        integrator.step(1)
        #move.apply(thermostate, sstate)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

now we can actually time the two integration schemes!!!!

In [10]:
mcmc_moves, splitting_thermodynamic_state, splitting_sampler_state = create_splitting_integrator(htf, constraint_tol = 1e-6)
context, langevin_thermodynamic_state, langevin_sampler_state, langevin_integrator   = create_langevin_integrator(htf, constraint_tol = 1e-6)

In [11]:
splitting_time = splitting_integrator_timer(mcmc_moves, splitting_thermodynamic_state, splitting_sampler_state, num_steps = 1000)
langevin_time = langevin_integrator_timer(langevin_integrator, num_steps = 1000)

In [12]:
print(f"splitting_time_integrator: {np.average(splitting_time)} +/- {np.std(splitting_time)}")
print(f"langevin time integrator: {np.average(langevin_time)} +/- {np.std(langevin_time)}")

splitting_time_integrator: 0.009207636833190917 +/- 0.0017017451857008273
langevin time integrator: 0.0032615756988525393 +/- 0.0010310747648119632


In [13]:
import simtk.openmm as openmm
with open(f"hybrid_system.xml", 'w') as f:
    hybrid_system = htf._hybrid_system
    f.write(openmm.openmm.XmlSerializer.serialize(hybrid_system))

now, we should see if we can regenerate the constraint problem by doing a transform from CCC --> CCCC (where the constraints should not change) and see if they run an order of magnitude faster

In [20]:
phase = 'vacuum'
molecule_setup = RelativeFEPSetup(ligand_input = 'test.smi', 
                                  old_ligand_index = 0, 
                                  new_ligand_index = 1, 
                                  forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'], 
                                  phases = [phase],
                                  protein_pdb_filename=None,
                                  receptor_mol2_filename=None, 
                                  pressure=1.0 * unit.atmosphere,
                                  temperature=300.0 * unit.kelvin, 
                                  solvent_padding=9.0 * unit.angstroms, 
                                  atom_map=None,
                                  hmass=4*unit.amus, 
                                  neglect_angles=False, 
                                  map_strength='default', 
                                  anneal_14s = False)
if phase == 'solvent':
    htf = HybridTopologyFactory(molecule_setup._solvent_topology_proposal,
                                molecule_setup._ligand_positions_old_solvated,
                                molecule_setup._ligand_positions_new_solvated)


    zero_state_error, one_state_error = validate_endstate_energies(htf._topology_proposal, htf, molecule_setup._solvated_added_valence_energy, molecule_setup._solvated_subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)
elif phase == 'vacuum':
    htf = HybridTopologyFactory(molecule_setup._vacuum_topology_proposal,
                            molecule_setup._vacuum_positions_old,
                            molecule_setup._vacuum_positions_new)


    zero_state_error, one_state_error = validate_endstate_energies(htf._topology_proposal, htf, molecule_setup._vacuum_added_valence_energy, molecule_setup._vacuum_subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)

INFO:relative_setup:			_hmass: 4 Da.

INFO:relative_setup:Handling files for ligands and indices...
INFO:relative_setup:Detected .smi format.  Proceeding...
INFO:relative_setup:	old smiles: CCC
INFO:relative_setup:	new smiles: CCCC
INFO:utils.openeye:molecule  does not have unique atom names. Generating now...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:utils.openeye:molecule  does not have unique atom names. Generating now...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:relative_setup:	successfully created old and new systems from smiles
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:propane
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule posi

added energy components: [('CustomBondForce', 0.9485138922949207), ('CustomAngleForce', 4.826887772263102), ('CustomTorsionForce', 5.287026256783554), ('CustomBondForce', 0.630847806517544)]
added energy components: [('CustomBondForce', 0.8531871065384696), ('CustomAngleForce', 0.10652772375353954), ('CustomTorsionForce', 0.004895238404749462), ('CustomBondForce', -0.7985065727667483)]
			HarmonicBondForce: 1.016248464203151
			HarmonicAngleForce: 0.21329026901376769
			PeriodicTorsionForce: 0.009820712992701786
			NonbondedForce: 2.0365928031768297
			AndersenThermostat: 0.0
added forces:3.2759522493864504
rp: 3.2759522493864504
			CustomBondForce: 0.16306135766468124
			HarmonicBondForce: 1.80170099883339
			CustomAngleForce: 0.10676254526022816
			HarmonicAngleForce: 4.933415496016642
			CustomTorsionForce: 0.004925474587952326
			PeriodicTorsionForce: 5.291921495188303
			NonbondedForce: 2.7250425685610127
			CustomNonbondedForce: -0.05760195886663859
			CustomBondForce: 0.0
			And

In [21]:
mcmc_moves, splitting_thermodynamic_state, splitting_sampler_state = create_splitting_integrator(htf, constraint_tol = 1e-6)
context, langevin_thermodynamic_state, langevin_sampler_state, langevin_integrator   = create_langevin_integrator(htf, constraint_tol = 1e-6)

In [22]:
#since we are adding a whole methyl group, we need to minimize
from perses.dispersed.feptasks import minimize
minimize(splitting_thermodynamic_state, splitting_sampler_state)
splitting_sampler_state.apply_to_context(context)


In [23]:
splitting_time = splitting_integrator_timer(mcmc_moves, splitting_thermodynamic_state, splitting_sampler_state, num_steps = 1000)
langevin_time = langevin_integrator_timer(langevin_integrator, num_steps = 1000)
print(f"splitting_time_integrator: {np.average(splitting_time)} +/- {np.std(splitting_time)}")
print(f"langevin time integrator: {np.average(langevin_time)} +/- {np.std(langevin_time)}")

splitting_time_integrator: 0.0022577104568481444 +/- 0.00015741323202442094
langevin time integrator: 0.00046755242347717286 +/- 5.627331036725344e-05


print the constraints

In [26]:
for i in range(htf._old_system.getNumConstraints()):
    print(htf._old_system.getConstraintParameters(i))

[0, 3, Quantity(value=0.10969000000000001, unit=nanometer)]
[0, 4, Quantity(value=0.10969000000000001, unit=nanometer)]
[0, 5, Quantity(value=0.10969000000000001, unit=nanometer)]
[1, 6, Quantity(value=0.10969000000000001, unit=nanometer)]
[1, 7, Quantity(value=0.10969000000000001, unit=nanometer)]
[1, 8, Quantity(value=0.10969000000000001, unit=nanometer)]
[2, 9, Quantity(value=0.10969000000000001, unit=nanometer)]
[2, 10, Quantity(value=0.10969000000000001, unit=nanometer)]


In [27]:
for i in range(htf._new_system.getNumConstraints()):
    print(htf._new_system.getConstraintParameters(i))

[0, 4, Quantity(value=0.10969000000000001, unit=nanometer)]
[0, 5, Quantity(value=0.10969000000000001, unit=nanometer)]
[0, 6, Quantity(value=0.10969000000000001, unit=nanometer)]
[1, 7, Quantity(value=0.10969000000000001, unit=nanometer)]
[1, 8, Quantity(value=0.10969000000000001, unit=nanometer)]
[1, 9, Quantity(value=0.10969000000000001, unit=nanometer)]
[2, 10, Quantity(value=0.10969000000000001, unit=nanometer)]
[2, 11, Quantity(value=0.10969000000000001, unit=nanometer)]
[3, 12, Quantity(value=0.10969000000000001, unit=nanometer)]
[3, 13, Quantity(value=0.10969000000000001, unit=nanometer)]
